# Query Above Options Timeseries

This notebook demonstrates how to query the `api_v1_above_options_timeseries` endpoint,
which returns probabilities and prices for "above" markets (daily, sampled every 5 minutes).

The endpoint supports two formats:
- **long**: One row per timestamp+horizon+strike combination
- **wide**: One row per timestamp, with columns for sorted strikes by position

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
load_dotenv("../.env.local")

from polybridge import PolybridgeClient
from datetime import datetime, timedelta, timezone

import pandas as pd

In [3]:
client = PolybridgeClient(api_key=os.getenv("POLYBRIDGE_API_KEY"))

## Long Format

Long format returns one row per timestamp+horizon+strike combination.
Columns: `timestamp_bucket`, `spot_price`, `spot_price_exchange`, `relative_horizon`, `strike_price`, `prob_above`

In [4]:
# Query with long format
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(hours=8)

result_long = client.fetch_above_options_timeseries(
    asset="BTC",
    start_ts=start_time.isoformat(),
    end_ts=end_time.isoformat(),
    format="long",  # Long format: one row per timestamp+horizon+strike
    horizon="daily"  # Optional, defaults to "daily"
)

# Convert to DataFrame
rows_long = result_long["rows"]
df_long = pd.DataFrame(rows_long)

In [5]:
# Display structure and sample data
print(f"Long format: {len(df_long)} rows")
print(f"\nColumns: {list(df_long.columns)}")
print(f"\nData types:")
print(df_long.dtypes)
print(f"\nFirst few rows:")
df_long.head(20)

Long format: 5500 rows

Columns: ['timestamp_bucket', 'spot_price', 'spot_price_exchange', 'relative_horizon', 'strike_price', 'prob_above']

Data types:
timestamp_bucket        object
spot_price             float64
spot_price_exchange     object
relative_horizon        object
strike_price           float64
prob_above             float64
dtype: object

First few rows:


,timestamp_bucket,spot_price,spot_price_exchange,relative_horizon,strike_price,prob_above
0,2025-11-05 15:20:00+00:00,103211.07,binance,next,102000.0,0.9270
1,2025-11-05 15:20:00+00:00,103211.07,binance,next,104000.0,0.1320
2,2025-11-05 15:20:00+00:00,103211.07,binance,next,106000.0,0.0085
3,2025-11-05 15:20:00+00:00,103211.07,binance,next,108000.0,0.0025
4,2025-11-05 15:20:00+00:00,103211.07,binance,next,110000.0,0.0005
5,2025-11-05 15:20:00+00:00,103211.07,binance,next,112000.0,0.0005
6,2025-11-05 15:20:00+00:00,103211.07,binance,next,114000.0,0.0005
7,2025-11-05 15:20:00+00:00,103211.07,binance,next,116000.0,0.0005
8,2025-11-05 15:20:00+00:00,103211.07,binance,next,118000.0,0.0005
9,2025-11-05 15:20:00+00:00,103211.07,binance,next,120000.0,0.0005


In [6]:
# Check unique values
print(f"Unique timestamps: {df_long['timestamp_bucket'].nunique()}")
print(f"Unique relative_horizons: {sorted(df_long['relative_horizon'].unique())}")
print(f"\nUnique strikes per horizon:")
for horizon in sorted(df_long['relative_horizon'].unique()):
    strikes = df_long[df_long['relative_horizon'] == horizon]['strike_price'].nunique()
    print(f"  {horizon}: {strikes} unique strikes")

Unique timestamps: 96
Unique relative_horizons: ['next', 'next+1', 'next+2', 'next+3', 'next+4', 'next+5', 'next+6']

Unique strikes per horizon:
  next: 13 unique strikes
  next+1: 12 unique strikes
  next+2: 11 unique strikes
  next+3: 11 unique strikes
  next+4: 11 unique strikes
  next+5: 13 unique strikes
  next+6: 11 unique strikes


## Wide Format

Wide format returns one row per timestamp.
Columns include:
- `timestamp_bucket`, `spot_price`, `spot_price_exchange`
- For each relative_horizon (e.g., "next", "next+1", ...):
  - `{horizon}_strike_1` through `{horizon}_strike_11` (sorted by strike price, ascending)
  - `{horizon}_prob_above_1` through `{horizon}_prob_above_11`

Example column names: `next_strike_1`, `next_prob_above_1`, `next_plus_1_strike_1`, etc.

In [7]:
# Query with wide format
result_wide = client.fetch_above_options_timeseries(
    asset="BTC",
    start_ts=start_time.isoformat(),
    end_ts=end_time.isoformat(),
    format="wide",  # Wide format: one row per timestamp
    horizon="daily"  # Optional, defaults to "daily"
)

# Convert to DataFrame
rows_wide = result_wide["rows"]
df_wide = pd.DataFrame(rows_wide)

In [8]:
# Display structure and sample data
print(f"Wide format: {len(df_wide)} rows")
print(f"\nTotal columns: {len(df_wide.columns)}")
print(f"\nColumn names (first 20):")
print(list(df_wide.columns)[:20])
print(f"\nFirst few rows:")
df_wide.head()

Wide format: 96 rows

Total columns: 157

Column names (first 20):
['timestamp_bucket', 'spot_price', 'spot_price_exchange', 'next_strike_1', 'next_prob_above_1', 'next_strike_2', 'next_prob_above_2', 'next_strike_3', 'next_prob_above_3', 'next_strike_4', 'next_prob_above_4', 'next_strike_5', 'next_prob_above_5', 'next_strike_6', 'next_prob_above_6', 'next_strike_7', 'next_prob_above_7', 'next_strike_8', 'next_prob_above_8', 'next_strike_9']

First few rows:


,timestamp_bucket,spot_price,spot_price_exchange,next_strike_1,next_prob_above_1,next_strike_2,next_prob_above_2,next_strike_3,next_prob_above_3,next_strike_4,...,next_plus_6_strike_7,next_plus_6_prob_above_7,next_plus_6_strike_8,next_plus_6_prob_above_8,next_plus_6_strike_9,next_plus_6_prob_above_9,next_plus_6_strike_10,next_plus_6_prob_above_10,next_plus_6_strike_11,next_plus_6_prob_above_11
0,2025-11-05 15:20:00+00:00,103211.07,binance,102000.0,0.927,104000.0,0.1320,106000.0,0.0085,108000.0,...,104000.0,0.455,106000.0,0.305,108000.0,0.190,110000.0,0.10,112000.0,0.0655
1,2025-11-05 15:25:00+00:00,103328.73,binance,102000.0,0.927,104000.0,0.1320,106000.0,0.0085,108000.0,...,104000.0,0.455,106000.0,0.305,108000.0,0.195,110000.0,0.10,112000.0,0.0655
2,2025-11-05 15:30:00+00:00,103344.58,binance,102000.0,0.950,104000.0,0.1675,106000.0,0.0055,108000.0,...,104000.0,0.455,106000.0,0.305,108000.0,0.195,110000.0,0.10,112000.0,0.0655
3,2025-11-05 15:35:00+00:00,103400.01,binance,102000.0,0.950,104000.0,0.1675,106000.0,0.0055,108000.0,...,104000.0,0.475,106000.0,0.325,108000.0,0.200,110000.0,0.10,112000.0,0.0655
4,2025-11-05 15:40:00+00:00,103545.64,binance,102000.0,0.950,104000.0,0.1550,106000.0,0.0055,108000.0,...,104000.0,0.475,106000.0,0.325,108000.0,0.200,110000.0,0.12,112000.0,0.0655


In [9]:
# Check how many strikes are present per horizon
strike_columns = [col for col in df_wide.columns if '_strike_' in col]
print(f"Total strike columns: {len(strike_columns)}")
print(f"\nSample strike columns:")
for col in sorted(strike_columns)[:20]:
    non_null = df_wide[col].notna().sum()
    print(f"  {col}: {non_null} non-null values")

Total strike columns: 77

Sample strike columns:
  next_plus_1_strike_1: 96 non-null values
  next_plus_1_strike_10: 96 non-null values
  next_plus_1_strike_11: 96 non-null values
  next_plus_1_strike_2: 96 non-null values
  next_plus_1_strike_3: 96 non-null values
  next_plus_1_strike_4: 96 non-null values
  next_plus_1_strike_5: 96 non-null values
  next_plus_1_strike_6: 96 non-null values
  next_plus_1_strike_7: 96 non-null values
  next_plus_1_strike_8: 96 non-null values
  next_plus_1_strike_9: 96 non-null values
  next_plus_2_strike_1: 96 non-null values
  next_plus_2_strike_10: 96 non-null values
  next_plus_2_strike_11: 96 non-null values
  next_plus_2_strike_2: 96 non-null values
  next_plus_2_strike_3: 96 non-null values
  next_plus_2_strike_4: 96 non-null values
  next_plus_2_strike_5: 96 non-null values
  next_plus_2_strike_6: 96 non-null values
  next_plus_2_strike_7: 96 non-null values
